# Public Transit Options Around A Location

---
Azure Maps Mobility service (http://bit.ly/332vWZv) allows functionality for public transit, bike share, and scooter share routing with multimodal trip planning options. For public transit, the service offers search for nearby stations, stops, and lines and provides information like scheduled and real-time public transit arrivals, list of stops and route guides, as well as detailed and multimodal step-by-step itineraries and service alerts. The mobility service also returns real-time and static availability information to car share vehicles and shared bikes and scooters. 

I have included a few scenarios in this walkthrough:

1. Geocoding to find the lat long for any address
2. Finding the MetroID for use in the Mobility API for the address you are analyzing
3. Using the Find Nearby Transit Options API

For Geocoding, Azure Maps does not have the same level of information and accuracy for all cities and countries/regions. Check this table to see if your region is covered: http://bit.ly/2LHMiAV


---

In [104]:
import os
import json
import time
import requests
import urllib.parse
from IPython.display import Image, display

Load Subscription Key for Azure Maps from Config File. If you dont have an Azure Maps Subscription Key, get yourself a free account here: http://bit.ly/2VV39SG

In [105]:
config_file = 'config.json'
key = json.load(open(config_file))['key']
subscriptionKey = key

The Azure Maps Search Service provides the ability for customers to find real world objects and their respective location. The Search Service provides for three major functions:
    1. Geocoding: Finding addresses, places, and landmarks
    2. Point of Interest (POI) Search: Finding businesses based on a location
    3. Reverse Geocoding: Finding addresses or cross streets based on a location
    
You can customize this for the address that you want analyze by replacing the address below:

In [98]:
addresstoAnalyze = "15127 NE 24th Street, Redmond, WA 98052"
locationDetails = requests.get("https://atlas.microsoft.com/search/address/json?subscription-key={}&api-version=1.0&query={}".format(subscriptionKey,addresstoAnalyze)).json()
latlong=locationDetails["results"][0]["position"]
latlong
currentLocation[0]=latlong["lat"]
currentLocation[1]=latlong["lon"]

The *Get Metro Area* API allows to request metro areas in which the Azure Maps Mobility Service is available. The service supports filtering results by country or coordinate location. Information returned includes Metro Area details such as metro Id, name and a representation of the metro area geometry in GeoJSON format.

In [106]:
metroAreaID = requests.get("https://atlas.microsoft.com/mobility/metroArea/id/json?subscription-key={}&api-version=1.0&queryType=position&query={}".format(subscriptionKey,str(currentLocation[0])+","+str(currentLocation[1]))).json()
metroID1 = metroAreaID["results"][0]["metroId"]
metroName1 = metroAreaID["results"][0]["metroName"]
print("The Metro ID for", metroName1, "is",metroID1)

The Metro ID for Seattle–Tacoma–Bellevue, WA is 522


In [107]:
transitOptions = requests.get("https://atlas.microsoft.com/mobility/transit/nearby/json?subscription-key={}&api-version=1.0&metroId={}&query={}&radius=1000".format(subscriptionKey,metroID1,str(currentLocation[0])+","+str(currentLocation[1]))).json()

In [108]:
transitOptionsF = json.dumps(transitOptions, indent=4)
print(transitOptionsF)

{
    "results": [
        {
            "id": "522---2060625",
            "type": "stop",
            "objectDetails": {
                "stopKey": "71370",
                "stopName": "NE 24th St & 152nd Ave NE",
                "stopCode": "71370",
                "mainTransitType": "Bus",
                "mainAgencyId": "522---5872",
                "mainAgencyName": "Metro Transit"
            },
            "position": {
                "latitude": 47.631485,
                "longitude": -122.137283
            },
            "viewport": {
                "topLeftPoint": {
                    "latitude": 47.632394,
                    "longitude": -122.138599
                },
                "btmRightPoint": {
                    "latitude": 47.630575,
                    "longitude": -122.135966
                }
            }
        },
        {
            "id": "522---2060612",
            "type": "stop",
            "objectDetails": {
                "stopKey": "71331",


In [109]:
print("Here are the public transit options: ")
for loc in range(len(transitOptions["results"])):
                print(loc+1,". ",transitOptions["results"][loc]["objectDetails"]["mainTransitType"],"(", transitOptions["results"][loc]["objectDetails"]["mainAgencyName"],")", ":", transitOptions["results"][loc]["objectDetails"]["stopName"])
                

Here are the public transit options: 
1 .  Bus ( Metro Transit ) : NE 24th St & 152nd Ave NE
2 .  Bus ( Metro Transit ) : Overlake Park & Ride
3 .  Bus ( Sound Transit ) : Overlake P&R
